# Кластеризация

В качестве меры крутости алгоритмов использовалась суммарная энтропия кластеров, а для DBScan ещё и количество кластеров. Путём перебора параметорв для DBScan удалось установить, что правильно разделяет цифры он при радиусе равном 4.795 и минимальном количестве соседей core point равном 50. Сравнение этих алгоритмов на подвыборке из 1000 цифр (т.к. на всех 60000 работает невероятно долго):

In [3]:
from kmeans import K_means
from dbscan import DBScan
from idx2numpy import convert_from_file
import numpy

images = numpy.reshape(convert_from_file("train-images.idx3-ubyte"), (60000, 784)).astype("float64")
labels = convert_from_file("train-labels.idx1-ubyte")
images = numpy.multiply(images, 1 / 255)


kmeans = K_means(10, 10)
kmeans.fit(numpy.array([images[i] for i in range(1000)]))
print("Summary k-means clusters entropy: %s" % kmeans.score(numpy.array([labels[i] for i in range(1000)])))

dbscan = DBScan(4.795, 50)
dbscan.fit(numpy.array([images[i] for i in range(1000)]))
print("Summary dbscan clusters entropy: %s" % dbscan.score(numpy.array([labels[i] for i in range(1000)])))
print("Clusters found by dbscan: %s " % dbscan.clusters())

Summary k-means clusters entropy: 0.331556966204
Summary dbscan clusters entropy: 0.995731623441
Clusters found by dbscan: 10 


Как видно, k-means кластеризовал данные гораздо лучше — суммарная энтропия в три раза меньше, однако dbscan не нужно заранее знать количество кластеров.

## Анализ алгоритмов

Из плюсов k-means можно отметить его достаточно быструю работу и очень быструю сходимость, по крайней мере на наборе MNIST. 
Из минусов — необходимость знать заранее количество кластеров.
Однако хотя dbscan и не нужно их задавать, он очень сильно зависит от параметров. Вот примеры, как незначительное 
варьирование радиуса и количества соседних точек влияет на точность:

In [8]:
from kmeans import K_means
from dbscan import DBScan
from idx2numpy import convert_from_file
import numpy

images = numpy.reshape(convert_from_file("train-images.idx3-ubyte"), (60000, 784)).astype("float64")
labels = convert_from_file("train-labels.idx1-ubyte")
images = numpy.multiply(images, 1 / 255)

print("Decreasing radius by 0.1.")
dbscan = DBScan(4.895, 50)
dbscan.fit(numpy.array([images[i] for i in range(1000)]))
print("Summary dbscan clusters entropy: %s" % dbscan.score(numpy.array([labels[i] for i in range(1000)])))
print("Clusters found by dbscan: %s \n" % dbscan.clusters())

print("Increasing radius by 0.1.")
dbscan = DBScan(4.695, 50)
dbscan.fit(numpy.array([images[i] for i in range(1000)]))
print("Summary dbscan clusters entropy: %s" % dbscan.score(numpy.array([labels[i] for i in range(1000)])))
print("Clusters found by dbscan: %s \n" % dbscan.clusters())

print("Decreasing neighbours by 20.")
dbscan = DBScan(4.795, 30)
dbscan.fit(numpy.array([images[i] for i in range(1000)]))
print("Summary dbscan clusters entropy: %s" % dbscan.score(numpy.array([labels[i] for i in range(1000)])))
print("Clusters found by dbscan: %s \n" % dbscan.clusters())

print("Increasing neighbours by 20.")
dbscan = DBScan(4.795, 70)
dbscan.fit(numpy.array([images[i] for i in range(1000)]))
print("Summary dbscan clusters entropy: %s" % dbscan.score(numpy.array([labels[i] for i in range(1000)])))
print("Clusters found by dbscan: %s " % dbscan.clusters())

Decreasing radius by 0.1.
Summary dbscan clusters entropy: 1.10418169756
Clusters found by dbscan: 7 

Increasing radius by 0.1.
Summary dbscan clusters entropy: 0.913794314102
Clusters found by dbscan: 15 

Decreasing neighbours by 20.
Summary dbscan clusters entropy: 0.995731623441
Clusters found by dbscan: 10 

Increasing neighbours by 20.
Summary dbscan clusters entropy: 1.10636847049
Clusters found by dbscan: 9 


## Асимптотика и память

Время работы dbscan — квадратичное в худшем случае, перед каждым вызовом expand приходится перебирать все данные в поисках соседей. Kd-деревья при таких размерностях не помогут, поэтому их использовать для обработки range query бесполезно. Память — O(N). 
Время работы kmeans — O(NlogN), память — O(N).